<a href="https://colab.research.google.com/github/usesnames/Reviews_analyzer/blob/main/Sentiment_analysis_intesa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt

# Acquisizione dati

In [ ]:
!pip install -qq google-play-scraper
#!pip install app-store-scraper
from google_play_scraper import Sort, reviews, app, reviews_all

     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 2.0MB 22.0MB/s 
     |████████████████████████████████| 1.2MB 40.5MB/s 
     |████████████████████████████████| 890kB 46.0MB/s 
     |████████████████████████████████| 3.2MB 50.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.3-cp37-none-any.whl size=114277 sha256=bd345389d88ea3950ca909c0dcc7413b44129b6bfac0538cb969b1a6b575e23b
  Stored in directory: /root/.cache/pip/wheels/c8/0d/fa/4e28cf045da4781344e7972befb2fdf306051b225bfc290187
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=60b8e0f4dfa73af9904285591ec56be138ce3bc8bc272636a24ae7503b1dafbf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
app_name = 'com.latuabancaperandroid'

Helper function that prints JSON objects a bit better:

In [ ]:
def print_json(json_object):
  json_str = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str
  )
  print(json_str)

In [ ]:
app_reviews = []

for score in list(range(1, 6)):
  rvs, _ = reviews(
    app_name,
    lang='it',
    country='it',
    sort=Sort.NEWEST,
    count= 4000, 
    filter_score_with=score
  )
  for r in rvs:
    r['appId'] = app_name
  app_reviews.extend(rvs)

KeyboardInterrupt: ignored

In [ ]:
print_json(app_reviews[0])

In [ ]:
len(app_reviews)

# Parametri

In [2]:
num_words = 10000 #20000

# Preprocessing

In [128]:
if not os.path.isfile('balanced_reviews.csv'):
  balanced_reviews_df = pd.DataFrame(app_reviews)
  balanced_reviews_df.to_csv('balanced_reviews.csv', index=None, header=True, sep='\t')
  print('generato file balanced_reviews.csv')

balanced_reviews_df = pd.read_csv('balanced_reviews.csv', sep='\t')
balanced_reviews_df.drop(labels=['userName', 'userImage', 'replyContent', 'repliedAt', 
                                 'at', 'appId', 'reviewCreatedVersion', 'reviewId'], 
                         axis=1, inplace=True)

In [129]:
balanced_reviews_df.tail()

,content,score,thumbsUpCount
19995,Funzionale e sicura anche con impronta Digital...,5,0
19996,Ok,5,0
19997,Molto funzionale,5,0
19998,Molto pratica e valida. Mi trovo benissimo,5,0
19999,"Ottima, molto comoda e di facile utilizzo",5,0


split

In [130]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(balanced_reviews_df['content'], balanced_reviews_df['score'], 
                                                    test_size=0.3, random_state=42, stratify=balanced_reviews_df['score'])
y_train = y_train.map(lambda num_stars: 1 if (num_stars == 5 or num_stars == 4) else 0)
y_test = y_test.map(lambda num_stars: 1 if (num_stars == 5 or num_stars == 4) else 0)

X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)

In [152]:
preprocessing_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=num_words, standardize='lower_and_strip_punctuation',
    split='whitespace', ngrams=(1,2), output_mode='binary', pad_to_max_tokens=True)
preprocessing_layer.adapt(np.array(X_train))

# Modello e training

In [153]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', max_iter=500)
clf.fit(preprocessing_layer(X_train).numpy(), y_train)
clf.score(preprocessing_layer(X_test).numpy(), y_test)

0.8871666666666667

In [155]:
clf.predict(preprocessing_layer(['non ottima']))

array([1])

#Serialization

In [156]:
# create a TF model with the same architecture
tf_model = tf.keras.models.Sequential()
tf_model.add(tf.keras.Input(shape=(num_words,)))
tf_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
# add preprocessing layer
model = tf.keras.models.Sequential()
model.add(preprocessing_layer)
model.add(tf_model)

# assign the parameters from sklearn to the TF model
tf_model.layers[0].weights[0].assign(clf.coef_.transpose())
tf_model.layers[0].bias.assign(clf.intercept_)

<tf.Variable 'UnreadVariable' shape=(1,) dtype=float32, numpy=array([-0.02202789], dtype=float32)>

In [157]:
predictions = clf.predict(preprocessing_layer(X_test).numpy())
tf_predictions = model(np.array(X_test)) 

flag="stesse predictions"
for el, tf_el in zip(predictions, tf_predictions.numpy()):
  if((el==1 and tf_el[0]<0.5) or (el==0 and tf_el[0]>0.5)):
    flag="predicitons divergono"
print(flag)

stesse predictions


In [ ]:
model.save('saved_model')

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model') 
converter.allow_custom_ops=True
tflite_model = converter.convert()

# Save the model.
with open('sentiment_model.tflite', 'wb') as f:
  f.write(tflite_model)

# Demo

In [163]:
demo_review = ['molto bella',"funziona", "non funziona", "c'è da migliorare", "così così", "non perfetta", "perfetta", "così così, non perfetta"]
demo_review2 = "bella e utile, ma troppi crash"

prediction = model.predict(demo_review)
for el in prediction:
  if el>0.6:
    print("recensione postiva", el)
  elif el<=0.6 and el>=0.4:
    print("recensione neutra", el)
  else:
    print('recensione negativa', el)

recensione postiva [0.8766632]
recensione neutra [0.5684332]
recensione negativa [0.07781635]
recensione neutra [0.40630516]
recensione neutra [0.54123056]
recensione neutra [0.47770432]
recensione postiva [0.90676206]
recensione neutra [0.4884223]


In [110]:
# model = tf.keras.Sequential([
#   preprocessing_layer,
#   tf.keras.layers.Reshape((1,12000)),
#   tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
#   #tf.keras.layers.MaxPooling1D(pool_size=2),
#   tf.keras.layers.Dropout(0.3),
#   tf.keras.layers.Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'),
#   #tf.keras.layers.MaxPooling1D(pool_size=2),
#   tf.keras.layers.Dropout(0.3),
#   #tf.keras.layers.LSTM(20),
#   tf.keras.layers.Dense(2, activation='softmax')
# ])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# history = model.fit(X_train, y_train, epochs=2,
#                     validation_data=(X_test, y_test))#, callbacks=tf.keras.callbacks.LearningRateScheduler(lr_schedule))